<a href="https://colab.research.google.com/github/EmTampz/Copra_YOLOv11_v12_Colab_Notebooks/blob/main/Copy_of_Copra_1_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# 📦 ENVIRONMENT SETUP for YOLOv11 Thesis (Colab)
# ==========================================================
# This installs only what is required for:
# - YOLOv11 object detection, classification, segmentation
# - Grid search tuning and visualization
# - Logging and performance tracking
# ==========================================================

# Core deep learning and YOLO library
!pip install -U ultralytics

# Core data handling, visualization, and ML utilities
!pip install pandas matplotlib seaborn pillow scikit-learn

# Optional (but recommended) for monitoring and logging
!pip install tensorboard tqdm

# Example Roboflow integration
!pip install roboflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
import torch, ultralytics, pandas as pd, matplotlib, seaborn, PIL, sklearn, tqdm, tensorboard

print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available  : {torch.cuda.is_available()}")
print(f"Ultralytics ver : {ultralytics.__version__}")
ultralytics.checks()

Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 39.3/112.6 GB disk)


In [ ]:
"""
gridsearch_tuning_yolo11.py
Automated grid search for YOLOv11 tasks (Detection → Segmentation → Classification)
- Downloads dataset via Roboflow API
- Performs hyperparameter grid search
- Saves timestamped CSV for each run
- Stores best model checkpoints
- Logs to TensorBoard
- Computes evaluation metrics
- Visualizes curves, confusion matrices, per-class metrics
- Zips all results for download
"""

import os, time, shutil, yaml, json, traceback, zipfile, glob, random
from datetime import datetime
from pathlib import Path
from itertools import product

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import ConfusionMatrixDisplay

from ultralytics import YOLO
from IPython.display import Image as IPyImage, display
from roboflow import Roboflow

# ---------------- USER CONFIG ----------------
# Initialize Roboflow API and download YOLOv11 dataset
rf = Roboflow(api_key="LS1Ohi2OQZ178OOqQ4zc")  # Replace with your key
project = rf.workspace("cv-opfhv").project("mor-v4-8itha")
version = project.version(1)
dataset = version.download("yolov11")  # YOLOv11 format
ROBOFLOW_DATA_YAML = os.path.join(dataset.location, "data.yaml")
print(f"✅ Dataset downloaded! YAML path: {ROBOFLOW_DATA_YAML}")

RESULTS_BASE = "/content/copra_yolo11_grid_results"
os.makedirs(RESULTS_BASE, exist_ok=True)

TASKS = ["detect", "seg", "cls"]
MODEL_MAP = {"detect":"yolo11n.pt","seg":"yolo11n-seg.pt","cls":"yolo11n-cls.pt"}
IMG_SIZE = 640
TENSORBOARD_ENABLE = True

# Grid search hyperparameters
GRID_EPOCHS = [100, 200, 300]
GRID_BATCHES = [16, 32, 64]
GRID_PATIENCES = [0, 50, 100]
GRID_OPTIMIZERS = ["SGD", "Adam"]
LR0 = 0.001

TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
GRID_CSV = os.path.join(RESULTS_BASE, f"grid_summary_{TIMESTAMP}.csv")

# ---------------- HELPER FUNCTIONS ----------------

def load_data_yaml(path):
    """Load dataset YAML file"""
    with open(path, "r") as f:
        return yaml.safe_load(f)

def iou_xyxy(box1, box2):
    """Compute IoU between two boxes [x1,y1,x2,y2]"""
    x1 = max(box1[0], box2[0]); y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2]); y2 = min(box1[3], box2[3])
    inter_w = max(0, x2 - x1); inter_h = max(0, y2 - y1)
    inter_area = inter_w * inter_h
    area1 = max(0, (box1[2]-box1[0])*(box1[3]-box1[1]))
    area2 = max(0, (box2[2]-box2[0])*(box2[3]-box2[1]))
    union = area1 + area2 - inter_area
    return inter_area/union if union>0 else 0.0

def yolo_label_to_xyxy(norm, w, h):
    """Convert normalized YOLO label to xyxy pixel coordinates"""
    xc, yc, bw, bh = norm
    xc *= w; yc *= h; bw *= w; bh *= h
    return [xc - bw/2, yc - bh/2, xc + bw/2, yc + bh/2]

def parse_label_file(label_path):
    """Parse YOLO txt label file into (class, [x,y,w,h]) tuples"""
    labels=[]
    if not os.path.exists(label_path): return labels
    with open(label_path,"r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts)>=5:
                labels.append((int(parts[0]), list(map(float,parts[1:5]))))
    return labels

def save_weights_copy(src_path, dest_dir, run_name):
    """Copy trained weights to saved_models folder"""
    os.makedirs(dest_dir, exist_ok=True)
    if src_path and os.path.exists(src_path):
        dest=os.path.join(dest_dir,f"{run_name}_{Path(src_path).name}")
        shutil.copy(src_path,dest)
        return dest
    return None

def ensure_tensorboard():
    """Enable TensorBoard in YOLO settings"""
    if TENSORBOARD_ENABLE:
        os.system("yolo settings tensorboard=True")

def display_training_curves(run_dir):
    """Display training curves (results.png)"""
    results_png = os.path.join(run_dir,"results.png")
    if os.path.exists(results_png):
        display(IPyImage(filename=results_png,width=600))

def display_confusion_matrix(cm, class_names, save_path=None):
    """Display and optionally save confusion matrix"""
    if cm is None: return
    disp = ConfusionMatrixDisplay(cm, display_labels=class_names + ["Unlabeled"])
    fig, ax = plt.subplots(figsize=(6,6))
    disp.plot(ax=ax, xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.tight_layout()
    if save_path: plt.savefig(save_path)
    plt.show(); plt.close()

def plot_per_class_metrics(metrics_dict, class_names, save_path=None):
    """Plot per-class Precision, Recall, F1 and optionally save"""
    x = np.arange(len(class_names)); width=0.2
    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(x-width, metrics_dict.get("precision",[0]*len(class_names)), width, label="Precision")
    ax.bar(x, metrics_dict.get("recall",[0]*len(class_names)), width, label="Recall")
    ax.bar(x+width, metrics_dict.get("f1",[0]*len(class_names)), width, label="F1-score")
    ax.set_xticks(x); ax.set_xticklabels(class_names,rotation=45,ha="right")
    ax.set_ylim(0,1); ax.set_ylabel("Score"); ax.set_title("Per-class Precision, Recall, F1"); ax.legend()
    plt.tight_layout()
    if save_path: plt.savefig(save_path)
    plt.show(); plt.close()

def zip_all_results(results_base, timestamp):
    """Zip all results: inference, weights, CSVs, plots"""
    zip_name=os.path.join(results_base,f"all_results_{timestamp}.zip")
    with zipfile.ZipFile(zip_name,'w',zipfile.ZIP_DEFLATED) as zipf:
        for folder in glob.glob(os.path.join(results_base,"**"), recursive=True):
            for root,_,files in os.walk(folder):
                for file in files:
                    file_path=os.path.join(root,file)
                    arcname=os.path.relpath(file_path,start=results_base)
                    zipf.write(file_path,arcname)
    print(f"\n📦 All results zipped: {zip_name}")
    return zip_name

# ---------------- SINGLE RUN ----------------
def run_single(task, base_model, cfg, results_base):
    """Run a single YOLO training with specified hyperparameters"""
    try:
        model = YOLO(base_model)
        ts=datetime.now().strftime("%Y%m%d_%H%M%S")
        run_name=f"{task}_grid_e{cfg['epochs']}_b{cfg['batch']}_p{cfg['patience']}_opt{cfg['optimizer']}_lr{cfg['lr0']}_{ts}"
        project_dir=os.path.join(results_base,task,run_name)
        os.makedirs(project_dir,exist_ok=True)

        print(f"\n>> Running {run_name}")
        t0=time.time()
        model.train(
            data=ROBOFLOW_DATA_YAML,
            epochs=cfg["epochs"], batch=cfg["batch"],
            imgsz=IMG_SIZE, optimizer=cfg["optimizer"],
            lr0=cfg["lr0"], patience=cfg["patience"],
            project=os.path.join(results_base,task),
            name=run_name,resume=False
        )
        elapsed=round(time.time()-t0,2)

        # Save trained weights
        weights_dir=os.path.join(project_dir,"weights")
        best_w=os.path.join(weights_dir,"best.pt"); last_w=os.path.join(weights_dir,"last.pt")
        chosen=best_w if os.path.exists(best_w) else (last_w if os.path.exists(last_w) else None)
        saved_model=save_weights_copy(chosen,os.path.join(results_base,"saved_models"),run_name)

        # Validate metrics
        val_results=model.val()
        metrics={}
        per_class_metrics={"precision":[],"recall":[],"f1":[]}
        try:
            metrics["mAP@0.5"]=float(val_results.box.map50)
            metrics["mAP@0.5:0.95"]=float(val_results.box.map)
            metrics["precision"]=float(val_results.box.precision)
            metrics["recall"]=float(val_results.box.recall)
            metrics["f1"]=float(val_results.box.f1)
            per_class_metrics["precision"]=val_results.box.pr.tolist() if hasattr(val_results.box,'pr') else []
            per_class_metrics["recall"]=val_results.box.re.tolist() if hasattr(val_results.box,'re') else []
            per_class_metrics["f1"]=val_results.box.f1s.tolist() if hasattr(val_results.box,'f1s') else []
        except Exception:
            metrics={"mAP@0.5":None,"mAP@0.5:0.95":None,"precision":None,"recall":None,"f1":None}

        # Limited inference & plots
        val_images_dir = Path(dataset.location)/"valid"/"images"
        sample_imgs=random.sample(glob.glob(str(val_images_dir)+"/*.jpg"),min(3,len(glob.glob(str(val_images_dir)+"/*.jpg"))))
        infer_folder=os.path.join(project_dir,"inference_results")
        os.makedirs(infer_folder,exist_ok=True)
        if sample_imgs: model.predict(source=sample_imgs, save=True, project=infer_folder, name="sample_preds")
        cm_path=os.path.join(project_dir,"confusion_matrix.png")
        pcm_path=os.path.join(project_dir,"per_class_metrics.png")
        data_meta=load_data_yaml(ROBOFLOW_DATA_YAML)
        class_names=data_meta.get("names",[])
        y_true=[]; y_pred=[]
        for img in sample_imgs:
            res_list=model.predict(img,imgsz=IMG_SIZE,conf=0.25,iou=0.45,stream=False)
            if not res_list: continue
            res=res_list[0]; preds=[]
            if hasattr(res,"boxes") and res.boxes.data is not None:
                for b in res.boxes.data.tolist(): preds.append(int(b[5]))
            label_path=Path(img).with_suffix(".txt"); gt=[]
            if label_path.exists():
                with open(label_path) as f: gt=[int(line.strip().split()[0]) for line in f]
            y_true.extend(gt); y_pred.extend(preds)
        cm=None; acc=None
        if y_true:
            nc=len(class_names); cm=np.zeros((nc+1,nc+1),dtype=int)
            for t,p in zip(y_true,y_pred):
                ti=nc if t==-1 else t; pi=nc if p==-1 else p; cm[ti,pi]+=1
            tn=cm[:nc,:nc]; acc=float(tn.trace()/tn.sum()) if tn.sum()>0 else None
            display_confusion_matrix(cm,class_names,save_path=cm_path)
            plot_per_class_metrics(per_class_metrics,class_names,save_path=pcm_path)

        return {
            "timestamp":ts,"task":task,"run_name":run_name,"saved_model":saved_model,
            "epochs":cfg["epochs"],"batch":cfg["batch"],"patience":cfg["patience"],"optimizer":cfg["optimizer"],"lr0":cfg["lr0"],
            "elapsed_seconds":elapsed,"mAP@0.5":metrics.get("mAP@0.5"),"mAP@0.5:0.95":metrics.get("mAP@0.5:0.95"),
            "precision":metrics.get("precision"),"recall":metrics.get("recall"),"f1":metrics.get("f1"),
            "accuracy":acc,"confusion_matrix":json.dumps(cm.tolist()) if cm is not None else None
        }
    except Exception as e:
        print("Error:",e); traceback.print_exc()
        return {"timestamp":datetime.now().strftime("%Y%m%d_%H%M%S"),"task":task,"run_name":None,"saved_model":None,
                "epochs":cfg.get("epochs"),"batch":cfg.get("batch"),"patience":cfg.get("patience"),
                "optimizer":cfg.get("optimizer"),"lr0":cfg.get("lr0"),"elapsed_seconds":None,
                "mAP@0.5":None,"mAP@0.5:0.95":None,"precision":None,"recall":None,"f1":None,"accuracy":None,
                "confusion_matrix":None,"error":str(e)}

# ---------------- GRID LOOP ----------------
if __name__=="__main__":
    ensure_tensorboard()
    data_meta=load_data_yaml(ROBOFLOW_DATA_YAML)
    class_names=data_meta.get("names",[])
    all_rows=[]
    total_runs=len(TASKS)*len(GRID_EPOCHS)*len(GRID_BATCHES)*len(GRID_PATIENCES)*len(GRID_OPTIMIZERS)
    print(f"Total runs planned: {total_runs}")

    # Run grid search
    for task in TASKS:
        base_model=MODEL_MAP[task]
        for e,b,p,opt in product(GRID_EPOCHS,GRID_BATCHES,GRID_PATIENCES,GRID_OPTIMIZERS):
            cfg={"epochs":e,"batch":b,"patience":p,"optimizer":opt,"lr0":LR0}
            rec=run_single(task,base_model,cfg,RESULTS_BASE)
            all_rows.append(rec)
            pd.DataFrame(all_rows).to_csv(GRID_CSV,index=False)
            print(f"Intermediate CSV saved ({len(all_rows)} runs) -> {GRID_CSV}")

    # Side-by-side mAP comparison
    if all_rows:
        fig,ax=plt.subplots(figsize=(10,5))
        tasks_labels=[r['run_name'] for r in all_rows]
        map50=[r['mAP@0.5'] for r in all_rows]
        map5095=[r['mAP@0.5:0.95'] for r in all_rows]
        width=0.35; x=np.arange(len(tasks_labels))
        ax.bar(x-width/2,map50,width,label="mAP@0.5")
        ax.bar(x+width/2,map5095,width,label="mAP@0.5:0.95")
        ax.set_xticks(x); ax.set_xticklabels(tasks_labels,rotation=45,ha="right")
        ax.set_ylabel("mAP"); ax.set_title("Side-by-side mAP Comparison (Grid Search)"); ax.legend()
        plt.tight_layout(); plt.show()

    # Zip all results (weights, inference, CSVs, plots)
    zip_all_results(RESULTS_BASE, TIMESTAMP)
    print("Grid search finished. Saved models:", os.path.join(RESULTS_BASE,"saved_models"))
